In [2]:
import requests
import pandas as pd
import numpy as np

In [24]:
#This is first running, season 2021, page 1
#Implement for loop to query data from previous years and more pages

#Use this API : Players statistics by league id
import requests

url = "https://api-football-v1.p.rapidapi.com/v3/players"

querystring = {"league":"39","season":"2021","page":"1"}

headers = {
    "X-RapidAPI-Host": "api-football-v1.p.rapidapi.com",
    "X-RapidAPI-Key": "d356c39fe7mshf3ec31c0312f244p17df85jsn2bc9fd933f3f"
}

response = requests.request("GET", url, headers=headers, params=querystring)

print(response)

<Response [200]>


In [14]:
import json
#from jsontraverse.parser import JsonTraverseParser
data = response.json()
data

{'get': 'players',
 'parameters': {'league': '39', 'page': '1', 'season': '2021'},
 'errors': [],
 'results': 20,
 'paging': {'current': 1, 'total': 39},
 'response': [{'player': {'id': 84,
    'name': 'Wesley Moraes',
    'firstname': 'Wesley',
    'lastname': 'Moraes Ferreira da Silva',
    'age': 26,
    'birth': {'date': '1996-11-26',
     'place': 'Juiz de Fora',
     'country': 'Brazil'},
    'nationality': 'Brazil',
    'height': '191 cm',
    'weight': '93 kg',
    'injured': False,
    'photo': 'https://media.api-sports.io/football/players/84.png'},
   'statistics': [{'team': {'id': 66,
      'name': 'Aston Villa',
      'logo': 'https://media.api-sports.io/football/teams/66.png'},
     'league': {'id': 39,
      'name': 'Premier League',
      'country': 'England',
      'logo': 'https://media.api-sports.io/football/leagues/39.png',
      'flag': 'https://media.api-sports.io/flags/gb.svg',
      'season': 2021},
     'games': {'appearences': 1,
      'lineups': 0,
      'minu

In [268]:
#Query all data from a year
data2019=[]
for i in range(30):
    url = "https://api-football-v1.p.rapidapi.com/v3/players"#

    querystring = {"league":"39","season":"2019","page":str(i)}

    headers = {
        "X-RapidAPI-Host": "api-football-v1.p.rapidapi.com",
        "X-RapidAPI-Key": "d356c39fe7mshf3ec31c0312f244p17df85jsn2bc9fd933f3f"
   }

    response = requests.request("GET", url, headers=headers, params=querystring)
    
    data2019.append(response.text)
    
data2019
#Good but have to pop last 9 pages(so from 28 and 37 inclusively , didn't work because exceeded rate per minute)

['{"get":"players","parameters":{"league":"39","page":"0","season":"2019"},"errors":[],"results":0,"paging":{"current":0,"total":33},"response":[]}',
 '{"get":"players","parameters":{"league":"39","page":"1","season":"2019"},"errors":[],"results":20,"paging":{"current":1,"total":33},"response":[{"player":{"id":160,"name":"M. Vorm","firstname":"Michel","lastname":"Vorm","age":38,"birth":{"date":"1983-10-20","place":"Nieuwegein","country":"Netherlands"},"nationality":"Netherlands","height":"183 cm","weight":"84 kg","injured":false,"photo":"https:\\/\\/media.api-sports.io\\/football\\/players\\/160.png"},"statistics":[{"team":{"id":47,"name":"Tottenham","logo":"https:\\/\\/media.api-sports.io\\/football\\/teams\\/47.png"},"league":{"id":39,"name":"Premier League","country":"England","logo":"https:\\/\\/media.api-sports.io\\/football\\/leagues\\/39.png","flag":"https:\\/\\/media.api-sports.io\\/flags\\/gb.svg","season":2019},"games":{"appearences":0,"lineups":0,"minutes":0,"number":null,"p

In [35]:
#complete year array
for i in range(30,34):
    url = "https://api-football-v1.p.rapidapi.com/v3/players"
    
    #YEAR TO CHANGE IN QUERY
    querystring = {"league":"39","season":"2019","page":str(i)}

    headers = {
        "X-RapidAPI-Host": "api-football-v1.p.rapidapi.com",
        "X-RapidAPI-Key": "d356c39fe7mshf3ec31c0312f244p17df85jsn2bc9fd933f3f"
    }

    response = requests.request("GET", url, headers=headers, params=querystring)
    
    data2019.append(response.text)
    
data2019

['{"get":"players","parameters":{"league":"39","page":"0","season":"2020"},"errors":[],"results":0,"paging":{"current":0,"total":37},"response":[]}',
 '{"get":"players","parameters":{"league":"39","page":"1","season":"2020"},"errors":[],"results":20,"paging":{"current":1,"total":37},"response":[{"player":{"id":114,"name":"Adrien S\\u00e9bastien Perruchet Silva","firstname":"Adrien S\\u00e9bastien","lastname":"Perruchet Silva","age":33,"birth":{"date":"1989-03-15","place":"Angoul\\u00eame","country":"France"},"nationality":"Portugal","height":"171 cm","weight":"72 kg","injured":false,"photo":"https:\\/\\/media.api-sports.io\\/football\\/players\\/114.png"},"statistics":[{"team":{"id":46,"name":"Leicester","logo":"https:\\/\\/media.api-sports.io\\/football\\/teams\\/46.png"},"league":{"id":39,"name":"Premier League","country":"England","logo":"https:\\/\\/media.api-sports.io\\/football\\/leagues\\/39.png","flag":"https:\\/\\/media.api-sports.io\\/flags\\/gb.svg","season":2020},"games":{"

In [161]:
from json import loads
#Data2020 is complete, next step is to decide how to model and keep which part of the data
#data2020Json = list(map(lambda i: data2020[i].json(),data2020))
tableau=[]
for i in range(34):
    tableau.append(i)
#Format data to json
jsonData2020=list(map(lambda i: loads(data2020[i]),tableau))
jsonData2020[34]

{'get': 'players',
 'parameters': {'league': '39', 'page': '37', 'season': '2020'},
 'errors': [],
 'results': 5,
 'paging': {'current': 37, 'total': 37},
 'response': [{'player': {'id': 17,
    'name': 'C. Pulisic',
    'firstname': 'Christian',
    'lastname': 'Pulisic',
    'age': 24,
    'birth': {'date': '1998-09-18', 'place': 'Hershey', 'country': 'USA'},
    'nationality': 'USA',
    'height': '172 cm',
    'weight': '69 kg',
    'injured': False,
    'photo': 'https://media.api-sports.io/football/players/17.png'},
   'statistics': [{'team': {'id': 49,
      'name': 'Chelsea',
      'logo': 'https://media.api-sports.io/football/teams/49.png'},
     'league': {'id': 39,
      'name': 'Premier League',
      'country': 'England',
      'logo': 'https://media.api-sports.io/football/leagues/39.png',
      'flag': 'https://media.api-sports.io/flags/gb.svg',
      'season': 2020},
     'games': {'appearences': 27,
      'lineups': 18,
      'minutes': 1735,
      'number': None,
     

In [234]:
#Create prototype array which will be column names in the df
proto= ["id"\
, "firstname"\
, "lastname"\
, "age"\
, "nationality"\
, "height"\
, "weight"\
, "injured"\
, "teamID"\
, "teamname"\
, "appearences"\
, "lineups"\
, "minutes"\
, "position"\
, "rating"\
, "captain"\
, "subIn"\
, "subOut"\
, "bench"\
, "totalShots"\
, "shotsOn"\
, "totalGoals"\
, "goalsConceded"\
, "goalAssists"\
, "goalSaves"\
, "totalPasses"\
, "keyPasses"\
, "passAccuracy"\
, "totalTackles"\
, "blocks"\
, "interceptions"\
, "duelsTotal"\
, "duelsWon"\
, "dribbleAttempts"\
, "dribblesSuccess"\
, "dribblesPast"\
, "foulsDrawn"\
, "foulsCommitted"\
, "cardsYel"\
, "cardsYelRed"\
, "cardsRed"\
, "penaltyWon"\
, "penaltyCommited"\
, "penaltyScored"\
, "penaltyMissed"\
, "penaltySaved"]

In [245]:
finalArr = []
#Keys to iterate through
playerKeys = ['id', 'firstname', 'lastname', 'age', 'nationality', 'height', 'weight', 'injured']
#Create a dict to optimize code
stat = {
    'team':['id','name'],
    'games':['appearences','lineups','minutes','position','rating','captain'],
    'substitutes':['in','out','bench'],
    'shots':['total','on'],
    'goals':['total','conceded','assists','saves'],
    'passes':['total','key','accuracy'],
    'tackles':['total','blocks','interceptions'],
    'duels':['total','won'],
    'dribbles':['attempts','success','past'],
    'fouls':['drawn','committed'],
    'cards':['yellow','yellowred','red'],
    'penalty':['won','commited','scored','missed','saved']
}

#To change before every extraction
iterable = 0
pagesNum = 37
finalPageItems = 5
for i in range(1,pagesNum+1):
    
    #To adjust for the last page
    if i == 37: 
        iterable = finalPageItems
    else:
        iterable = 20
        
    for j in range(iterable):
        to_fill = []
        for pkey in playerKeys:
            to_fill.append(jsonData2020[i]['response'][j]['player'][pkey])
        for key in stat:
            for innerkey in stat[key]:
                to_fill.append(jsonData2020[i]['response'][j]['statistics'][0][key][innerkey])        
        finalArr.append(to_fill)

In [250]:
finalArr
#len(proto)

[[114,
  'Adrien Sébastien',
  'Perruchet Silva',
  33,
  'Portugal',
  '171 cm',
  '72 kg',
  False,
  46,
  'Leicester',
  0,
  0,
  0,
  'Midfielder',
  None,
  False,
  0,
  0,
  0,
  None,
  None,
  0,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  0,
  0,
  0,
  None,
  None,
  None,
  None,
  None],
 [458,
  'Mbaye',
  'Diagne',
  31,
  'Senegal',
  '191 cm',
  '86 kg',
  False,
  60,
  'West Brom',
  16,
  14,
  1193,
  'Attacker',
  '6.720000',
  False,
  2,
  6,
  4,
  24,
  10,
  3,
  0,
  2,
  None,
  235,
  9,
  9,
  4,
  1,
  1,
  223,
  89,
  16,
  10,
  None,
  23,
  17,
  2,
  0,
  0,
  None,
  None,
  0,
  0,
  None],
 [518,
  'Meritan',
  'Shabani',
  22,
  'Germany',
  '185 cm',
  '78 kg',
  False,
  39,
  'Wolves',
  0,
  0,
  0,
  'Midfielder',
  None,
  False,
  0,
  0,
  3,
  None,
  None,
  0,
  0,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  

In [251]:
#TO NOTE: Year in query is season XXXX/XXXX+1
#Example if 2020 is past as param, the data returned is from the season 2020/2021
#Create a 2d array
arr = np.array(finalArr)
arr

array([[114, 'Adrien Sébastien', 'Perruchet Silva', ..., None, None,
        None],
       [458, 'Mbaye', 'Diagne', ..., 0, 0, None],
       [518, 'Meritan', 'Shabani', ..., 0, 0, None],
       ...,
       [2294, 'Willian', 'Borges da Silva', ..., 0, 0, None],
       [18897, 'DeAndre Roselle', 'Yedlin', ..., 0, 0, None],
       [19549, 'Antonee', 'Robinson', ..., 0, 0, None]], dtype=object)

In [253]:
#create the df
df = pd.DataFrame(arr, columns = proto)
df

,id,firstname,lastname,age,nationality,height,weight,injured,teamID,teamname,...,foulsDrawn,foulsCommitted,cardsYel,cardsYelRed,cardsRed,penaltyWon,penaltyCommited,penaltyScored,penaltyMissed,penaltySaved
0,114,Adrien Sébastien,Perruchet Silva,33,Portugal,171 cm,72 kg,False,46,Leicester,...,None,None,0,0,0,None,None,None,None,None
1,458,Mbaye,Diagne,31,Senegal,191 cm,86 kg,False,60,West Brom,...,23,17,2,0,0,None,None,0,0,None
2,518,Meritan,Shabani,22,Germany,185 cm,78 kg,False,39,Wolves,...,None,None,0,0,0,None,None,0,0,None
3,642,Sergio Leonel,Agüero del Castillo,33,Argentina,173 cm,70 kg,False,50,Manchester City,...,7,2,0,0,0,None,None,1,1,None
4,1200,Branislav,Ivanović,37,Serbia,185 cm,89 kg,False,60,West Brom,...,4,11,2,0,0,None,None,0,0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
720,17,Christian,Pulisic,24,USA,172 cm,69 kg,False,49,Chelsea,...,33,13,2,0,0,None,None,0,0,None
721,890,Faustino Marcos Alberto,Rojo,32,Argentina,187 cm,85 kg,False,33,Manchester United,...,None,None,0,0,0,None,None,None,None,None
722,2294,Willian,Borges da Silva,34,Brazil,175 cm,77 kg,False,42,Arsenal,...,21,5,0,0,0,None,None,0,0,None
723,18897,DeAndre Roselle,Yedlin,29,USA,172 cm,75 kg,False,34,Newcastle,...,4,4,0,0,0,None,None,0,0,None


In [261]:
df['season'] = '2020/2021'

In [266]:
#Export the df to a csv
df.to_csv('data2020&2021.csv')